In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse, r2_score,explained_variance_score
import sklearn.model_selection

from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split

from math import sqrt
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import env
import wrangle

# Step 1. Wrangle the data

- The get_df function returns a clean dataframe by performing the following tasks:
> 1. A SQL query of the Zillow database produces results that include only single family homes purchased between May and June 2017. 
> 2. Zeros and null values were filtered out and removed. 
> 3. Any non essential columns were dropped
> 4. A new column was added designating the county where each home is located. All homes in the data set are located in the state of California


In [2]:
df = wrangle.get_df()
df

,square_feet,taxes,home_value,bedroom_count,bathroom_count,County
0,1458.0,2319.90,136104.0,3.0,2.0,Los Angeles
1,1421.0,543.69,35606.0,2.0,1.0,Los Angeles
2,2541.0,9819.72,880456.0,4.0,3.0,Orange
3,1650.0,7673.19,614000.0,3.0,2.0,Los Angeles
4,693.0,3267.47,274237.0,2.0,1.0,Los Angeles
...,...,...,...,...,...,...
14985,2526.0,5718.72,458903.0,5.0,3.0,Orange
14986,2735.0,1465.88,115387.0,5.0,3.0,Orange
14987,1536.0,3519.78,297097.0,3.0,2.0,Los Angeles
14988,2655.0,8065.50,746963.0,5.0,3.0,Orange


- To double check that there are no zeros or null values before we start working with the data, run the following asserts and NA count

In [3]:
assert (df['square_feet'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"
assert (df['taxes'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"
assert (df['home_value'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"
assert (df['bedroom_count'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"
assert (df['bathroom_count'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"

In [4]:
df.isnull().sum()

square_feet       0
taxes             0
home_value        0
bedroom_count     0
bathroom_count    0
County            0
dtype: int64